### Labels

In [13]:
import tensorflow as tf
import pandas as pd
import numpy as np

attr = pd.read_csv('../list_attr_celeba.csv')
features = ['image_id', 'Attractive', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair', 'Male', 'Oval_Face', 'Receding_Hairline', 'Smiling']

attr_selected = attr.loc[:, features].iloc[:50000,:]
attr_selected.shape

(50000, 11)

### Images

In [11]:
import zipfile
from PIL import Image
from io import BytesIO

# Lazy loading
def image_generator(zip_file_path):
    i = 1
    with zipfile.ZipFile(zip_file_path, 'r') as archive:
        for entry in archive.infolist():
            if entry.filename.endswith(('.jpg', '.jpeg', '.png')):
                if i <=50000:
                    with archive.open(entry, 'r') as file:
                        img_data = BytesIO(file.read())
                        img = Image.open(img_data)
                        i += 1
                        yield entry.filename, img

zip_file_path = '../archive.zip'
generator = image_generator(zip_file_path)

50000


In [41]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Modify the image_generator to include resizing and data augmentation
def image_generator(zip_file_path, target_size=(64, 64), batch_size=32):
    data_gen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    i = 1
    with zipfile.ZipFile(zip_file_path, 'r') as archive:
        while i <= 50000:
            batch_filenames = []
            batch_images = []
            for _ in range(batch_size):
                entry = archive.getinfo(f'img_align_celeba/img_align_celeba/{i:06}.jpg')
                with archive.open(entry, 'r') as file:
                    img_data = BytesIO(file.read())
                    img = Image.open(img_data)
                    img = img.resize(target_size)
                    batch_filenames.append(entry.filename.split("/")[-1])
                    batch_images.append(np.array(img))
                    i += 1
            batch_images = np.array(batch_images)
            batch_labels = attr_selected[attr_selected['image_id'].isin(batch_filenames)].iloc[:, 1:].to_numpy()
            # print(batch_images.shape)
            # print(batch_labels)
            yield data_gen.flow(batch_images, batch_labels, batch_size=batch_size)


In [42]:
zip_file_path = '../archive.zip'
train_generator = image_generator(zip_file_path)

In [55]:
# Create an iterator from the generator
train_iterator = iter(train_generator)

# Get the first batch of images and labels
first_batch = next(train_iterator)
# print(first_batch)
batch_images_labels = first_batch
print(batch_images_labels[0][0].shape) # batch, size, channel

# print(batch_images[0])
# Display the first image and its corresponding labels
import matplotlib.pyplot as plt

# plt.imshow(batch_images[0])
# plt.show()
# print("Labels:", batch_labels[0])


(32, 64, 64, 3)


In [22]:
train_generator.

generator

In [ ]:
def img_preprocess(generator, attr_selected, target_size, image_datagen):
    for filename, img in generator:
    # Process each image (e.g., resize, normalize, and pass to the model)
        filename = filename.split("/")[-1]
        img = img.resize(target_size)
        img_array = np.array(img)
        img_array = image_datagen.random_transform(img_array)
        img_array /= 255.0

In [12]:
set(img_size)

{(178, 218)}